In [1]:
from aniso_MLMD.trainer import load_datasets
import pandas as pd
import os
import torch
import numpy as np

In [2]:
data_path ="/home/marjan/Documents/code-base/ml_datasets/pps_20_new/"

In [3]:
train_df = pd.read_pickle(os.path.join(data_path, 'train.pkl'))

In [3]:
train_dataloader, valid_dataloader, test_dataloader = load_datasets(data_path, 5)

In [6]:
for i, ((position, q, orientation_R, neighbor_list), target_force, target_torque,
                energy) in enumerate(
                train_dataloader):
    break

In [7]:
batch_size = position.shape[0]
N_particles = position.shape[1]

neighbor_list = neighbor_list.reshape(batch_size, N_particles, -1,
                                      neighbor_list.shape[-1])[:, :, :, 1] # (B, N, N_neighbors)

In [35]:
N_neighbors = neighbor_list.shape[-1]

NN_extended = neighbor_list.unsqueeze(-1).expand((batch_size, N_particles, N_neighbors, 3))

In [47]:
NN_poistions = torch.gather(x[:, :, None, :].expand((-1, -1, N_neighbors, -1)), 1, NN_extended)

In [48]:
neighbor_list[0, 0]

tensor([ 2,  3,  4,  7,  8,  9, 12, 13, 14, 19])

In [50]:
position[0][2]

tensor([ 0.5594, -4.8215,  6.6095])

In [43]:
Nn_poistions[0, 0]

tensor([[-7.2435,  2.3273,  5.7778],
        [-2.8743, -2.5427,  7.4393],
        [-0.7604, -6.3282, -4.7953],
        [-5.6185, -2.6832,  7.3069],
        [-1.7478, -7.4556,  4.2400],
        [-7.8379, -4.9343, -0.1946],
        [-5.8722, -6.8171,  3.2381],
        [-2.9921, -6.0537,  6.0638],
        [-6.2570,  1.3165, -4.0611],
        [-6.8157,  4.2841, -4.7450]])